In [21]:
##DQN prototyping

import gym
import numpy as np
import tensorflow as tf
from collections import deque, namedtuple

In [28]:
c = deque(maxlen=100)

In [29]:
Exp = namedtuple("experience", field_names=['s', 'a', 'r', 's_p'])

In [30]:
c.append(Exp(1,2,3,5))
c.append(Exp(2,3,4,5))


In [31]:
c

deque([experience(s=1, a=2, r=3, s_p=5), experience(s=2, a=3, r=4, s_p=5)])

In [32]:
indices = [0,1]

#  = 
s, a, r, s_p = zip([c[i] for i in indices])

ValueError: not enough values to unpack (expected 4, got 2)

In [40]:
state = np.array([])
actions = np.array([])
rewards = np.array([])
state_p = np.array([])
for i in range(len(c)):
 
    s, a, r, s_p = c[i]
    state = np.append(state,s) 
    actions = np.append(actions, a) 
    rewards = np.append(rewards, r) 
    state_p = np.append(state_p, s_p)

0
1


In [ ]:
class ExperienceReplay():
    def __init__(self, size):
        self.buffer = deque(maxlen=size)
        
    def __len__(self):
        return len(self.buffer)
    
    def add(self, experience):
        self.buffer.append(experience)
        
    def sample(self, sample_size):
        state = np.array([])
        actions = np.array([])
        rewards = np.array([])
        state_p = np.array([])
        ##get sample of indices
        if sample_size > len(self.buffer):
            for i in range(len(self.buffer)):
                s, a, r, s_p = c[i]
                state = np.append(state,s) 
                actions = np.append(actions, a) 
                rewards = np.append(rewards, r) 
                state_p = np.append(state_p, s_p)
            return state, actions, rewards, state_p
        else:
            indices = np.random.choice(len(self.buffer), 
                                       sample_size, 
                                       replace=False)
            for i in indices:
                s, a, r, s_p = c[i]
                state = np.append(state,s) 
                actions = np.append(actions, a) 
                rewards = np.append(rewards, r) 
                state_p = np.append(state_p, s_p)
                return state, actions, rewards, state_p
            
            